In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time
from sklearn.model_selection import StratifiedKFold

In [15]:
test = pd.read_csv('pima-indians-diabetes_test.data', header = None)
train = pd.read_csv('pima-indians-diabetes_train.data', header = None)

In [16]:
col = ['pregnant', 'glucose', 'bloodpressure', 'skin', 'ilsulin', 'bmi' ,'diabetes_faimily','age', 'class']
test.columns = col
train.columns = col

In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   pregnant          100 non-null    int64  
 1   glucose           100 non-null    int64  
 2   bloodpressure     100 non-null    int64  
 3   skin              100 non-null    int64  
 4   ilsulin           100 non-null    int64  
 5   bmi               100 non-null    float64
 6   diabetes_faimily  100 non-null    float64
 7   age               100 non-null    int64  
 8   class             100 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 7.2 KB


In [18]:
X_train = train.iloc[:,:8]
y_train = train.iloc[:,8]

In [19]:
X_test = test.iloc[:,:8]
y_test = test.iloc[:,8]

In [20]:
X_train

,pregnant,glucose,bloodpressure,skin,ilsulin,bmi,diabetes_faimily,age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
663,9,145,80,46,130,37.9,0.637,40
664,6,115,60,39,0,33.7,0.245,40
665,1,112,80,45,132,34.8,0.217,24
666,4,145,82,18,0,32.5,0.235,70


In [21]:
def build_model(num_layers, num_nod, drop_rate, lr):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape = 8))
    model.add(keras.layers.BatchNormalization())
    for iterzz in range(0,num_layers):
        model.add(keras.layers.Dense(num_nod, activation = 'selu', kernel_initializer = 'lecun_normal'))
        model.add(keras.layers.AlphaDropout(rate = drop_rate))
        model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1, activation = 'sigmoid', kernel_initializer = 'lecun_normal'))
    optimizer = keras.optimizers.SGD(learning_rate = 0.001, momentum = 0.9, nesterov = True)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return(model)

In [22]:
def k_fold(X, y, num_layers, num_nod, drop_rate, lr, k):
    folds = StratifiedKFold(n_splits = k, shuffle = True, random_state = 201600177)
    acc = []
    for train_idx, val_idx in folds.split(X,y):
        X_cv_train = X.iloc[train_idx,]
        y_cv_train = y[train_idx]
        X_cv_valid = X.iloc[val_idx,]
        y_cv_valid = y[val_idx]
        model = None
        model = build_model(num_layers, num_nod, drop_rate, lr)
        callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 20)
        model.fit(X_cv_train, y_cv_train, epochs = 10000, callbacks = callback, batch_size = 128, verbose = 0)
        acc.append(model.evaluate(X_cv_valid, y_cv_valid, verbose = 0)[1])
    return(sum(acc)/len(acc))

In [23]:
def exapnd_grid(num_layers, num_nods, drop_rate, lr):
    return(np.array([(aa, ab, round(ac,5), round(ad,5)) for aa in num_layers for ab in num_nods for ac in drop_rate for ad in lr]))

In [24]:
grid = exapnd_grid(np.arange(1,4), np.arange(20,210,30), np.arange(0.05,0.2,0.05), [0.0001, 0.001, 0.01, 0.1])

In [27]:
def grid_search(X, y, gr, k):
    accl = []
    start = time.time()
    for i,g in enumerate(gr):
        accl.append(k_fold(X, y, int(g[0]), int(g[1]), g[2], g[3], k))
        print(f'{i}/{len(gr)-1}')
    return([gr[accl.index(max(accl))], max(accl), time.time() - start])

In [28]:
result = grid_search(X_train, y_train, grid, 10)

0/335
1/335
2/335
3/335
4/335
5/335
6/335
7/335
8/335
9/335
10/335
11/335
12/335
13/335
14/335
15/335
16/335
17/335
18/335
19/335
20/335
21/335
22/335
23/335
24/335
25/335
26/335
27/335
28/335
29/335
30/335
31/335
32/335
33/335
34/335
35/335
36/335
37/335
38/335
39/335
40/335
41/335
42/335
43/335
44/335
45/335
46/335
47/335
48/335
49/335
50/335
51/335
52/335
53/335
54/335
55/335
56/335
57/335
58/335
59/335
60/335
61/335
62/335
63/335
64/335
65/335
66/335
67/335
68/335
69/335
70/335
71/335
72/335
73/335
74/335
75/335
76/335
77/335
78/335
79/335
80/335
81/335
82/335
83/335
84/335
85/335
86/335
87/335
88/335
89/335
90/335
91/335
92/335
93/335
94/335
95/335
96/335
97/335
98/335
99/335
100/335
101/335
102/335
103/335
104/335
105/335
106/335
107/335
108/335
109/335
110/335
111/335
112/335
113/335
114/335
115/335
116/335
117/335
118/335
119/335
120/335
121/335
122/335
123/335
124/335
125/335
126/335
127/335
128/335
129/335
130/335
131/335
132/335
133/335
134/335
135/335
136/335
137/335
138/33

In [29]:
print(len(grid))
print(result)

336
[array([2.e+00, 2.e+01, 1.e-01, 1.e-04]), 0.7799638211727142, 13005.478254318237]


In [42]:
tuned_model = build_model(int(result[0][0]), int(result[0][1]), result[0][2], result[0][3])
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 20)
tuned_model.fit(X_train, y_train, epochs = 10000, callbacks = callback, batch_size = 128)

Epoch 1/10000
6/6 [==============================] - 1s 6ms/step - loss: 0.9046 - accuracy: 0.4506
Epoch 2/10000
6/6 [==============================] - 0s 6ms/step - loss: 0.8829 - accuracy: 0.4536
Epoch 3/10000
6/6 [==============================] - 0s 6ms/step - loss: 0.8259 - accuracy: 0.4865
Epoch 4/10000
6/6 [==============================] - 0s 6ms/step - loss: 0.8100 - accuracy: 0.5195
Epoch 5/10000
6/6 [==============================] - 0s 6ms/step - loss: 0.8121 - accuracy: 0.5045
Epoch 6/10000
6/6 [==============================] - 0s 6ms/step - loss: 0.7499 - accuracy: 0.5284
Epoch 7/10000
6/6 [==============================] - 0s 6ms/step - loss: 0.7386 - accuracy: 0.5674
Epoch 8/10000
6/6 [==============================] - 0s 6ms/step - loss: 0.7202 - accuracy: 0.5793
Epoch 9/10000
6/6 [==============================] - 0s 6ms/step - loss: 0.6870 - accuracy: 0.6048
Epoch 10/10000
6/6 [==============================] - 0s 6ms/step - loss: 0.6876 - accuracy: 0.6093
Epoch 11/

In [43]:
tuned_model.evaluate(X_test,y_test)

4/4 [==============================] - 0s 2ms/step - loss: 0.4993 - accuracy: 0.7800


[0.49931854009628296, 0.7799999713897705]

In [44]:
keras.__version__

'2.5.0'

In [45]:
tf.version.VERSION

'2.5.0'

In [51]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11634299366819952112,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7020688179
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13694658950082813815
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:26:00.0, compute capability: 8.6"]